These are the beginnings of snowflake, a browser based way to access linux containers. I want minimal dependencies, for maximal easy maintenence. 

In [ ]:
# First, I need a way to store used ip addresses, to avoid conflicts. 

import shelve # I just need a key value store that is more secure than a dictionary stored entirely in ram

addr = shelve.open('addr') # creates persistent, extremely lightweight database

addr['data'] = {'one':1, 'two': 2}

print(addr['data']['one'])

addr.close

# only works on first run. Hmmm. Do I really need this to be persistent for more than one run? Or do I need it to be persistent across reboots?  
# I wish there was a database library that autocreated a database if it didn't exist, and used the existing one if it did.

In [ ]:
# I might need a way to get ip addressed, but I think I can simply use above code to store used ones, and avoid conflict

In [ ]:
# Now, I need a way to generate the relevant data for the containers. I will use nix's fromJSON feature to get the data from JSON files generated by python

import json






In [17]:
# Actually I have a better Idea. I will use nixs ability to evaluate pure functions and the import feature of nix, which imports text files.

import os

exampleVar = 1

with open("test", "w") as test:
    test.write(str(exampleVar))

# this file can then be evaluated with nix's import function

